In [ ]:
## OLD VERSIONS

In [ ]:
def scores_reg(name, X, y, ypr, exclude=['Adj_R2']):

    MAE = metrics.mean_absolute_error(y, ypr)
    MSE = metrics.mean_squared_error(y, ypr)
    RMSE = np.sqrt(MSE)
    R2 = metrics.r2_score(y, ypr)
    n = y.shape[0] # nb of observations
    p = X.shape[1] # nb of indep features
    Adj_R2 = 1-(1-R2)*(n-1)/(n-p-1)
    MAPE = 100*np.mean(np.abs((y-ypr)/(y+1e-10)))
    MSPE = 100*np.mean(np.square((y-ypr)/y+1e-10))
    expl_var = metrics.explained_variance_score(y, ypr)

    dict_metrics = {'MAE': MAE, 'MSE': MSE, 'RMSE': RMSE,
                    'R2': R2, 'expl_var': expl_var, 'Adj_R2': Adj_R2,
                    'MAPE': MAPE, 'MSPE': MSPE}
    
    li_n_metrics = [n for n in dict_metrics.keys() if n not in exclude]
    li_metrics = [dict_metrics[n] for n in li_n_metrics]
    ser = pd.Series(li_metrics, index = li_n_metrics, name=name)

    return ser

In [ ]:
def cv_scores_reg(name, pipe, X, y, cv=5, adj_r2=False):
    res = pd.Series()
    cv_scoring = ['neg_root_mean_squared_error', 'r2']

    cv_scores = cross_validate(pipe, X, y, scoring=cv_scoring,                       
                               cv=cv, return_train_score=True, verbose=1)
    n = y.shape[0]/cv # n: nb of observations
    p = X.shape[1] # p: nb of indep features
    Adj_R2 = (1-(1-cv_scores['test_r2'])*(n-1)/(n-p-1)).mean()
    if adj_r2:
        res = pd.Series({'mean_CV_te_RMSE': -cv_scores['test_neg_root_mean_squared_error'].mean(),
	                     'mean_CV_te_R2': cv_scores['test_r2'].mean(),
	                     'mean_CV_te_adjR2': Adj_R2},
                         name = name)
    else:
        res = pd.Series({'mean_CV_te_RMSE': -cv_scores['test_neg_root_mean_squared_error'],
	                     'mean_CV_te_R2': cv_scores['test_r2']}, name = name)
    return res

In [ ]:
def get_append_scores(name_reg, pipe, Xte, yte, df_res=None, cv=6):
    if df_res is None:
        df_res = pd.DataFrame(dtype = 'object')
    df_res_mod = pd.DataFrame(dtype = 'object')
    ypr = pipe.predict(Xte)
    ser = scores_reg(name_reg, Xte, yte, ypr).astype('object')
    ser = ser.append(cv_scores_reg(name_reg, pipe, Xte, yte, cv=cv).astype('object'))
    df_res_mod = pd.concat([df_res,ser.to_frame()],1)
    return df_res_mod

In [ ]:
def scv_perf_fetcher(name_reg, scv,
                     cv_results=True,
                     Xte=None, yte=None,
                     test_set=False,  exclude=['Adj_R2'],
                     cross_val=False, cv_test=6):
    
    df_res = pd.DataFrame(dtype = 'object')
    # 1 | best hyperparams and scores obtained during searchCV (training, testing)
    if cv_results:
        df_best_res = pd.DataFrame(scv.cv_results_).loc[scv.best_index_].astype('object')
        df_res.at['best_params', name_reg] = str(df_best_res['params'])
        li_index = df_best_res.index[df_best_res.index.str.startswith(('mean_', 'std_'))]
        li_index = move_cat_containing(li_index, ['train', 'test', 'score', 'fit'])
        for i in li_index:
            df_res.at[i, name_reg] = df_best_res[i]

    # 2 | score of the model with best params on test set (optional)
    if test_set:
        ypr = scv.best_estimator_.predict(Xte)
        res = scores_reg(name_reg, Xte, yte, ypr,
                        exclude=['Adj_R2']).astype('object')
        df_res = df_res.append(res.to_frame())

    # 3 | mean cv score of the model with best params on test set (optional)
    if cross_val:
        res = cv_scores_reg(name_reg, scv.best_estimator_,
                            Xte, yte, cv=cv_test).astype('object')
        df_res = df_res.append(res.to_frame())

    return df_res
